# 1. Etração dos dados

### imports

In [12]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

### Criando conexão os bancos de dados (transacional e DW)

In [4]:
DB_HOST_ORIGEM = "localhost"
DB_NAME_ORIGEM = "crm_transacional_db"
DB_USER_ORIGEM = "crm_user"
DB_PASSWORD_ORIGEM = "crm_password"
DB_PORT_ORIGEM = "5432"

conexao_origem_str = (
    f"postgresql://{DB_USER_ORIGEM}:{DB_PASSWORD_ORIGEM}@"
    f"{DB_HOST_ORIGEM}:{DB_PORT_ORIGEM}/{DB_NAME_ORIGEM}"
)
engine_origem = create_engine(conexao_origem_str)


DB_HOST_DW = "localhost"
DB_NAME_DW = "dw_db"
DB_USER_DW = "dw_user"
DB_PASSWORD_DW = "dw_password"
DB_PORT_DW = "5433"

conexao_dw_str = (
    f"postgresql://{DB_USER_DW}:{DB_PASSWORD_DW}@"
    f"{DB_HOST_DW}:{DB_PORT_DW}/{DB_NAME_DW}"
)
engine_dw = create_engine(conexao_dw_str)

### Consumindo dados

In [5]:
query_clientes = "SELECT * FROM CLIENTE"
query_categoria_clientes = "SELECT * FROM CATEGORIA_CLIENTE"
query_categoria_produto = "SELECT * FROM CATEGORIA_PRODUTO"
query_fornecedores = "SELECT * FROM FORNECEDORES"
query_item_vendas = "SELECT * FROM ITEM_VENDAS"
query_localidade = "SELECT * FROM LOCALIDADE"
query_lojas = "SELECT * FROM LOJAS"
query_produtos = "SELECT * FROM PRODUTO"
query_produto_fornecedor = "SELECT * FROM CLIENTE"
query_promocoes = "SELECT * FROM PROMOCOES"
query_vendas = "SELECT * FROM VENDAS"
query_vendedor = "SELECT * FROM VENDEDOR"


df_clientes = pd.read_sql(query_clientes, engine_origem)
df_categoria_clientes = pd.read_sql(query_categoria_clientes, engine_origem)
df_categoria_produto = pd.read_sql(query_categoria_produto, engine_origem)
df_fornecedores = pd.read_sql(query_fornecedores, engine_origem)
df_item_vendas = pd.read_sql(query_item_vendas, engine_origem)
df_localidade = pd.read_sql(query_localidade, engine_origem)
df_lojas = pd.read_sql(query_lojas, engine_origem)
df_produtos = pd.read_sql(query_produtos, engine_origem)
df_produto_fornecedor = pd.read_sql(query_produto_fornecedor, engine_origem)
df_promocoes = pd.read_sql(query_promocoes, engine_origem)
df_vendas = pd.read_sql(query_vendas, engine_origem)
df_vendedor = pd.read_sql(query_vendedor, engine_origem)

### Verificando nome das colunas

In [18]:
dataframes = {
        "df_clientes": df_clientes,
        "df_categoria_clientes": df_categoria_clientes,
        "df_categoria_produto": df_categoria_produto,
        "df_fornecedores": df_fornecedores,
        "df_item_vendas": df_item_vendas,
        "df_localidade": df_localidade,
        "df_lojas": df_lojas,
        "df_produtos": df_produtos,
        "df_produto_fornecedor": df_produto_fornecedor,
        "df_promocoes": df_promocoes,
        "df_vendas": df_vendas,
        "df_vendedor": df_vendedor,
}

for df_name, df in dataframes.items():
        print(f"--- DataFrame: {df_name} ---")
        print("\nColunas:")
        print(df.columns.tolist())
        print("\n" + "="*50 + "\n")

--- DataFrame: df_clientes ---

Colunas:
['id_cliente', 'nome_cliente', 'idade', 'genero', 'id_categoria_cliente', 'id_localidade']


--- DataFrame: df_categoria_clientes ---

Colunas:
['id_categoria_cliente', 'nome_categoria_cliente']


--- DataFrame: df_categoria_produto ---

Colunas:
['id_categoria_produto', 'nome_categoria_produto']


--- DataFrame: df_fornecedores ---

Colunas:
['id_fornecedor', 'nome_fornecedor', 'pais_origem']


--- DataFrame: df_item_vendas ---

Colunas:
['id_venda', 'id_produto', 'qtd_vendida', 'preco_venda', 'id_promocao_aplicada']


--- DataFrame: df_localidade ---

Colunas:
['id_localidade', 'cidade', 'estado', 'regiao']


--- DataFrame: df_lojas ---

Colunas:
['id_loja', 'nome_loja', 'gerente_loja', 'cidade', 'estado']


--- DataFrame: df_produtos ---

Colunas:
['id_produto', 'nome_produto', 'id_categoria_produto']


--- DataFrame: df_produto_fornecedor ---

Colunas:
['id_cliente', 'nome_cliente', 'idade', 'genero', 'id_categoria_cliente', 'id_localidade']

# 2. Transformação dos dados

### Dimensão clientes

In [22]:
df_clientes

,id_cliente,nome_cliente,idade,genero,id_categoria_cliente,id_localidade
0,1,Ana Sophia Silveira,48,F,3,105
1,2,Dr. Davi Miguel Almeida,63,None,4,105
2,3,Sr. Luiz Otávio Farias,28,Masculino,1,105
3,4,Ana Beatriz Monteiro,58,Feminino,10,105
4,5,Lorenzo Duarte,23,F,10,105
...,...,...,...,...,...,...
7995,7996,Sra. Hellena Ferreira,58,Outro,10,66
7996,7997,Mariah Souza,51,F,2,52
7997,7998,Antônio Peixoto,56,Masculino,5,89
7998,7999,Isis Rocha,72,F,10,105


In [21]:
# Idades que são nulas, menores que 1 e maiores que 100 recebem a média das idades como valor. Idade passa a ser inteiro.

idade_para_media = df_clientes['idade'].copy()

idade_para_media[idade_para_media < 1] = np.nan
idade_para_media[idade_para_media > 100] = np.nan
media_idades_validas = idade_para_media.mean()
media_idades_validas = round(media_idades_validas)

condicao_idade_invalida = (df_clientes['idade'] < 1) | \
                          (df_clientes['idade'] > 100) | \
                          (df_clientes['idade'].isnull())

df_clientes.loc[condicao_idade_invalida, 'idade'] = media_idades_validas
df_clientes['idade'] = df_clientes['idade'].astype(int)


